# ID3 Decision Tree

## Условие
* Реализирайте алгоритъма за класификационно дърво ID3
* Използвайте ***кросвалидация*** за изчисляване на точността на модела върху обучаващото множество. 

* За избягване на ***overfitting*** използвайте константа **K** -- минимален брой на обучаващи примери в множеството. 

? друг подход за избягване на ***overfittting*** + сравняване на резултата?  
? bonus: ***Random Forest*** ?

In [270]:
import math
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo, dotdict
from typing import Dict, List, Set, Tuple
from dataclasses import dataclass, field

In [271]:
# fetch dataset 

breast_cancer: dotdict = fetch_ucirepo(id=14)

X: pd.core.frame.DataFrame = breast_cancer.data.features 
y: pd.core.frame.DataFrame = breast_cancer.data.targets

values: List[str] = [X[name].unique() for idx, name in enumerate(X.columns)]

X: np.ndarray = X.values
y: np.ndarray = y.values
pos_target: str = "recurrence-events"



In [272]:

Split = Dict[str, "Data"]

class Data:
    def __init__(self, 
                 X_train_ref,
                 y_train_ref,
                 feature: int = -1,  
                 data: List[int] = None):
        
        self.X_train_ref = X_train_ref
        self.y_train_ref = y_train_ref 
        self.num_pos: int = 0
        self.feature: int = feature
        self.data: List[int] = data if data else []
        self._count_pos()

    def _count_pos(self) -> None:
        for idx in self.data:
            if self.y_train_ref[idx] == pos_target:
                self.num_pos += 1
    @property
    def prop_predict(self) -> bool:
        return self.prop_pos > 0.5

    @property
    def prop_pos(self) -> float:
        return self.num_pos / len(self.data)
    @property
    def zero_entropy(self) -> bool:
        return self.num_pos == len(self) or self.num_pos == 0 
    
    @property
    def full_entropy(self) -> bool:
        return self.num_pos == (len(self) - self.num_pos) 
    
    @property
    def entropy(self) -> float:
        if self.zero_entropy:
            return 0

        num_neg: int = (len(self) - self.num_pos) 
        prop_neg: float = num_neg / len(self)

        return -self.num_pos * math.log(self.prop_pos) - num_neg * math.log(prop_neg)

    @property
    def gini(self) -> float:
        return 2 * self.pos_prop * (1 - self.pos_prop)

    def __len__(self) -> int:
        return len(self.data)

    def insert_idx(self, idx: int) -> None:
        self.data.append(idx)
        if self.y_train_ref[idx] == pos_target:
            self.num_pos += 1

    def split_on_attribute(self, feature: int) -> Split:
        spl: Split = {val: Data(self.X_train_ref, 
                                self.y_train_ref, 
                                feature=feature) for val in values[feature]}
        
        for idx in self.data:
            val: str = self.X_train_ref[idx][feature]
            spl[val].insert_idx(idx)

        return spl

    def __repr__(self) -> str:
        return repr(self.data)

dt = Data(X, y, data=list(range(len(X))))
# spl: Split = dt.split_on_attribute(0)
# print(dt.data)
# print(dt.num_pos)
# print(dt.prop_pos)
# print(dt.entropy)
# for key, value in spl.items():
#     print(f"{key}: {value.entropy}")


In [273]:
class DecisionTree:
    K: int = 20
    D: int = 2
    def __init__(self, 
                 attributes: List[bool], 
                 data: Data=Data, depth=-1) -> None:
        self.data: Data = data
        self.children: Dict[str, DecisionTree] = {}
        self.attributes: List[bool] = attributes

        if depth != -1:
            self._build_children_depth(depth)
        else:
            self._build_children()
    
    def feature_id(self) -> int:
        return self.data.feature

    def information_gain(self, spl: Split) -> float:
        sum_term: float = sum((len(val) / len(self.data)) * val.entropy for _, val in spl.items())
        return self.data.entropy - sum_term

    def get_best_split(self) -> Tuple[int, Split]:
        best: Split = {}
        best_score: float = float("-inf")
        feature: int = -1
        for i in range(len(self.attributes)):
            if not self.attributes[i]:
                spl: Split = self.data.split_on_attribute(i)
                ig: float = self.information_gain(spl)
                if ig > best_score:
                    best_score = ig
                    best = spl
                    feature = i
        return feature, best

    @property
    def leaf(self) -> bool:
        return False if self.children else True

    def predict(self, x: np.ndarray) -> bool:
        if self.leaf:
            return self.data.prop_predict
        
        for val, child in self.children.items():
            if x[self.attribute] == val:
                if child.leaf and child.data.full_entropy:
                    return self.data.prop_predict
                return child.predict(x)

    @property
    def attribute(self) -> str:
        return self.data.feature

    @property
    def all_used(self) -> bool:
        return sum(self.attributes) == len(self.attributes)

    def _build_children(self) -> None:
        if self.data.zero_entropy or self.all_used or len(self.data) < DecisionTree.K:
            return

        feature, best_split = self.get_best_split()
        self.data.feature = feature
        self.attributes[feature] = True
    
        for val, data in best_split.items():
            self.children[val] = DecisionTree(self.attributes, data, depth=-1)

    def _build_children_depth(self, depth: int) -> None:
        if self.data.zero_entropy or self.all_used or depth > DecisionTree.D:
            return

        feature, best_split = self.get_best_split()
        self.data.feature = feature
        self.attributes[feature] = True
    
        for val, data in best_split.items():
            self.children[val] = DecisionTree(self.attributes, data, depth=depth + 1)


attributes = [False for _ in range(X.shape[1])]
n = DecisionTree(attributes, Data(X[100:], y[100:], data=list(range(len(X[100:])))))

# for i in range(len(X)):


# print("age", n.information_gain(spl))
# print(n.get_best_split())
# print(n.children)
# print(n.predict(X[0]))




## Measuring Accuracy

In [274]:
from sklearn.model_selection import  train_test_split, KFold


def measure_accuracy_num_left_pruning(X, y):
    """
    10-fold cross validation
    """
    kf = KFold(n_splits=10, shuffle=True)

    accs = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        attributes = [False for _ in range(X.shape[1])]
        n = DecisionTree(attributes, Data(X_train, y_train, data=list(range(len(X_train)))))

        mistakes = 0
        for i in range(len(X_test)):
            temp = y_test[i] == pos_target
            if temp != n.predict(X_test[i]):
                mistakes += 1
        accs.append(1 - mistakes / len(X_test)) 
 
    return sum(accs) / len(accs)

accs = [measure_accuracy_num_left_pruning(X, y) for _ in range(10)]
for elem in accs: 
    print(elem)

print(f"Average: {sum(accs) / len(accs): .2f}")

0.6668719211822661
0.6578817733990148
0.6467980295566502
0.6710591133004926
0.668472906403941
0.6466748768472905
0.6535714285714286
0.6610837438423646
0.670935960591133
0.6854679802955664
Average:  0.66


In [275]:
def measure_accuracy_depth_pruning(X, y):
    kf = KFold(n_splits=10, shuffle=True)

    accs = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        attributes = [False for _ in range(X.shape[1])]
        n = DecisionTree(attributes, Data(X_train, y_train, data=list(range(len(X_train)))), depth=0)

        mistakes = 0
        for i in range(len(X_test)):
            temp = y_test[i] == pos_target
            if temp != n.predict(X_test[i]):
                mistakes += 1
        accs.append(1 - mistakes / len(X_test)) 

    return sum(accs) / len(accs)


accs = [measure_accuracy_depth_pruning(X, y) for _ in range(10)]
for elem in accs: 
    print(elem)

print(f"Average: {sum(accs) / len(accs): .2f}")


0.6848522167487684
0.6955665024630543
0.705665024630542
0.6887931034482759
0.6788177339901479
0.6784482758620691
0.7030788177339902
0.6992610837438423
0.6782019704433498
0.6887931034482758
Average:  0.69


* Сравнение със ***sklearn***

In [277]:
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.preprocessing import OrdinalEncoder

breast_cancer: dotdict = fetch_ucirepo(id=14)
encoder = OrdinalEncoder()

X: pd.core.frame.DataFrame = breast_cancer.data.features 
y: pd.core.frame.DataFrame = breast_cancer.data.targets

X_encoded = pd.DataFrame(encoder.fit_transform(X), columns=X.columns)
y_encoded = pd.DataFrame(encoder.fit_transform(y), columns=y.columns)

print(f"entropy criterion, min_samples_split=2, max_depth=None")
clf = tree.DecisionTreeClassifier(random_state=0, 
                                  criterion="entropy")
accs = cross_val_score(clf, X_encoded, y_encoded, cv=10)
print(f"Average: {accs.mean(): .2f}")

print(f"gini criterion, min_samples_split=2, max_depth=None")
clf = tree.DecisionTreeClassifier(random_state=0, 
                                  criterion="gini")
accs = cross_val_score(clf, X_encoded, y_encoded, cv=10)
print(f"Average: {accs.mean(): .2f}")

print(f"entropy criterion, min_samples_split={DecisionTree.K}, max_depth=None")
clf = tree.DecisionTreeClassifier(random_state=0, 
                                  criterion="entropy", 
                                  min_samples_split=DecisionTree.K)
accs = cross_val_score(clf, X_encoded, y_encoded, cv=10)
print(f"Average: {accs.mean(): .2f}")


print(f"entropy criterion, min_samples_split = 2, max_depth={DecisionTree.D}")
clf = tree.DecisionTreeClassifier(random_state=0, 
                                  criterion="entropy", 
                                  max_depth=DecisionTree.D)
accs = cross_val_score(clf, X_encoded, y_encoded, cv=10)
print(f"Average: {accs.mean(): .2f}")

entropy criterion, min_samples_split=2, max_depth=None
Average:  0.66
gini criterion, min_samples_split=2, max_depth=None
Average: 0.687807881773399
entropy criterion, min_samples_split=20, max_depth=None
Average: 0.6732758620689655
entropy criterion, min_samples_split = 2, max_depth=2
Average: 0.7088669950738917
